In [17]:
import pymongo
from pymongo import MongoClient
from jsonschema import validate
import json
from dataclasses import dataclass, asdict, field,make_dataclass
from dataclasses_json import config, dataclass_json
from py2neo import NodeMatcher, Graph, Node, Relationship, RelationshipMatcher
from scalpl import Cut
from faker import Faker
import random

fake = Faker()
graph = Graph(password="12345")
node = Node()

ConnectionUnavailable: Connection has been closed

In [18]:
@dataclass_json
@dataclass(order=True, frozen=True)
class graphfield_d:
    labels: dict = field(default_factory=list) # node/link type
    score_ttp: str = " " # ??
    mitre_attack: dict = field(default_factory=list, metadata=config(field_name="mitre-attack"))
    ids: dict = field(default_factory=list) # property id

@dataclass_json            
@dataclass(order=True, frozen=True)
class node:
    id: str # uniwue node id
    parentId: str = " "# node grouping id
    type: str = "node"
    t: str = "" # node display title
    d: dict = field(default_factory=graphfield_d)
    tag: int = 0
        
@dataclass_json            
@dataclass(order=True, frozen=True)
class link:
    dt: list = field(default_factory=list)
    a2: str = "true"
    id1: str = " "
    id2: str = " "
    id: str = " "
    type: str = "link"
    t: str = "" # node display title
    d: dict = field(default_factory=graphfield_d)

@dataclass_json            
@dataclass(order=True, frozen=True)
class graphData_l2:
    type: str = "LinkChart"
    items: list = field(default_factory=list)
        
@dataclass_json            
@dataclass(order=True, frozen=True)
class graphData_l1:
    _id: str = "000"
    type: str = "graphData"
    data: dict = field(default_factory=graphData_l2)
        
@dataclass_json            
@dataclass(order=True, frozen=True)
class property_l2:
    ComputerName: str
    Account_Name: str
    Creator_Process_Name: str
    Creator_Process_ID: str
    EventCode: str
    New_Process_ID: str
    New_Process_Name: str
    OriginatingComputerName: str
    RecordNumber: str
    _time: str
    Process_Command_Line: str
    custodio_time: int
    path: str
    Title: list = field(default_factory=list)
        
@dataclass_json            
@dataclass(order=True, frozen=True)
class property_l1:
    _id: str = "000"
    type: str = "property"
    id: str = " "
    Properties: dict = field(default_factory=property_l2)

In [19]:
def create_node(id, title, labelslist, mitrelist):
    graphfield_d_template = graphfield_d(labels=labelslist, mitre_attack=mitrelist)
    node_template = node(id= id, t=title, d=graphfield_d_template)
    node_data = node_template.to_dict()
    return node_data

def create_link(id, datetimelist, id1, id2, linkname, propertyidlist, mitrelist):
    graphfield_d_template = graphfield_d(ids=propertyidlist, mitre_attack=mitrelist)
    link_template = link(
        id= id,
        dt=datetimelist,
        id1=id1,
        id2=id2,
        t= linkname,
        d= graphfield_d_template)
    link_data = link_template.to_dict()
    return link_data

def create_graphdata(items_data):
    graphData_l2_template = graphData_l2(items=items_data)
    graphData_l1_template = graphData_l1(data = graphData_l2_template)
    graphData_l1_data = graphData_l1_template.to_dict()
    return graphData_l1_data

def create_property(id,
                    ComputerName, 
                    Account_Name, 
                    Creator_Process_Name, 
                    Creator_Process_ID, 
                    EventCode, 
                    New_Process_ID, 
                    New_Process_Name,
                   OriginatingComputerName,
                   RecordNumber,
                   _time,
                   Process_Command_Line,
                   custodio_time,
                   path,
                   Title):
    property_l2_template = property_l2(
        ComputerName = ComputerName,
        Account_Name=  Account_Name,
        Creator_Process_Name=  Creator_Process_Name,
        Creator_Process_ID=  Creator_Process_ID,
        EventCode= EventCode,
        New_Process_ID=  New_Process_ID,
        New_Process_Name=  New_Process_Name,
        OriginatingComputerName=  OriginatingComputerName,
        RecordNumber=  RecordNumber,
        _time=  _time,
        Process_Command_Line=  Process_Command_Line,
        custodio_time= custodio_time,
        path=  path,
        Title=  Title)
    property_l1_template = property_l1(id = "1", Properties= property_l2_template)
    property_l1_data = property_l1_template.to_dict()
    return property_l1_data

In [20]:
nodedata = [("servicehost.exe", "process", "T1045"),
           ("someprocess.exe", "process", "T1045")
           ]

nodepack = []
for i, data in enumerate(nodedata, start=1):
    temp = create_node(i, title=data[0], labelslist = data[1], mitrelist = data[2])
    nodepack.append(temp)

In [21]:
nodepack

[{'id': 1,
  'parentId': ' ',
  'type': 'node',
  't': 'servicehost.exe',
  'd': {'labels': 'process',
   'score_ttp': ' ',
   'mitre-attack': 'T1045',
   'ids': []},
  'tag': 0},
 {'id': 2,
  'parentId': ' ',
  'type': 'node',
  't': 'someprocess.exe',
  'd': {'labels': 'process',
   'score_ttp': ' ',
   'mitre-attack': 'T1045',
   'ids': []},
  'tag': 0}]

In [22]:
link1 = create_link("1", [12222, 212121], "1", "2", "created", ["101"], ["T1022"])

In [23]:
property1 = create_property(
        id = "1",
        ComputerName = "DESKTOP-GNSLAB2.STELLA.LOCAL\t",
        Account_Name=  "DESKTOP-GNSLAB2$",
        Creator_Process_Name=  " C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunkd.exe  ",
        Creator_Process_ID=  " 0x728  ",
        EventCode= "4688",
        New_Process_ID=  "  0x14c8  ",
        New_Process_Name=  " C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunk-winprintmon.exe  ",
        OriginatingComputerName=  "10.0.5.112\t",
        RecordNumber=  "968797\t",
        _time=  "2020-05-08 11:01:11",
        Process_Command_Line=  " \"C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunk-winprintmon.exe\"  ",
        custodio_time=  1588935671000,
        path=  " C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunk-winprintmon.exe  ",
        Title=  [
          "splunk-winprintmon.exe"
        ])
property1

{'_id': '000',
 'type': 'property',
 'id': '1',
 'Properties': {'ComputerName': 'DESKTOP-GNSLAB2.STELLA.LOCAL\t',
  'Account_Name': 'DESKTOP-GNSLAB2$',
  'Creator_Process_Name': ' C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunkd.exe  ',
  'Creator_Process_ID': ' 0x728  ',
  'EventCode': '4688',
  'New_Process_ID': '  0x14c8  ',
  'New_Process_Name': ' C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunk-winprintmon.exe  ',
  'OriginatingComputerName': '10.0.5.112\t',
  'RecordNumber': '968797\t',
  '_time': '2020-05-08 11:01:11',
  'Process_Command_Line': ' "C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunk-winprintmon.exe"  ',
  'custodio_time': 1588935671000,
  'path': ' C:\\Program Files\\SplunkUniversalForwarder\\bin\\splunk-winprintmon.exe  ',
  'Title': ['splunk-winprintmon.exe']}}

In [11]:
graphdata1 = create_graphdata([node1, node2, link1])
data = [
    graphdata1,
    property1
]
data

NameError: name 'node1' is not defined

# Synthetic Data

In [24]:
fakemitre = ["T1022", "T9433"]

In [25]:
fakegraphdata = []

for i in range(5):
    temp = create_node(i, title=fake.file_name(), labelslist = [fake.word()], mitrelist = [random.choice(fakemitre)])
    fakegraphdata.append(temp)
    
for i in range(5):
    datatime = [int(fake.date_time_this_year().timestamp()) for x in range(5)]
    temp = create_link(i, datatime, random.randrange(0, 5), random.randrange(0, 5), "created", ["10"+str(i)], [random.choice(fakemitre)])
    fakegraphdata.append(temp)

for i in range(5):
    temp = create_property(
        id = i,
        ComputerName = fake.sentence(),
        Account_Name=  fake.sentence(),
        Creator_Process_Name=  fake.sentence(),
        Creator_Process_ID=  " 0x728  ",
        EventCode= "4688",
        New_Process_ID=  "  0x14c8  ",
        New_Process_Name=  fake.sentence(),
        OriginatingComputerName=  "10.0.5.112\t",
        RecordNumber=  "968797\t",
        _time=  fake.date_time_this_year().isoformat(),
        Process_Command_Line= fake.sentence(),
        custodio_time=  1588935671000,
        path=  fake.sentence(),
        Title=  [
          fake.file_name()
        ])
    fakegraphdata.append(temp)
    
fakegraphdata

[{'id': 0,
  'parentId': ' ',
  'type': 'node',
  't': 'happy.odp',
  'd': {'labels': ['think'],
   'score_ttp': ' ',
   'mitre-attack': ['T9433'],
   'ids': []},
  'tag': 0},
 {'id': 1,
  'parentId': ' ',
  'type': 'node',
  't': 'authority.mp3',
  'd': {'labels': ['traditional'],
   'score_ttp': ' ',
   'mitre-attack': ['T1022'],
   'ids': []},
  'tag': 0},
 {'id': 2,
  'parentId': ' ',
  'type': 'node',
  't': 'interview.avi',
  'd': {'labels': ['decade'],
   'score_ttp': ' ',
   'mitre-attack': ['T9433'],
   'ids': []},
  'tag': 0},
 {'id': 3,
  'parentId': ' ',
  'type': 'node',
  't': 'want.bmp',
  'd': {'labels': ['fish'],
   'score_ttp': ' ',
   'mitre-attack': ['T1022'],
   'ids': []},
  'tag': 0},
 {'id': 4,
  'parentId': ' ',
  'type': 'node',
  't': 'stop.pptx',
  'd': {'labels': ['quality'],
   'score_ttp': ' ',
   'mitre-attack': ['T1022'],
   'ids': []},
  'tag': 0},
 {'dt': [1614398580, 1624525276, 1612449551, 1611084958, 1628573150],
  'a2': 'true',
  'id1': 0,
  'id2'

# NEO4J

In [14]:
graphdata = data[0]
scalpeldata = Cut(graphdata)

graph_nodelist = [x.data for x in scalpeldata.all('data.items') if x['type'] == 'node']
graph_linklist = [x.data for x in scalpeldata.all('data.items') if x['type'] == 'link']
graph_properties = data[1:]

TypeError: Cannot access key 'data' in path 'data.items': the element must be a dictionary or a list but is of type '<class 'str'>'.

In [15]:
graph_properties

NameError: name 'graph_properties' is not defined

In [53]:
graph_properties

[['{\'ComputerName\': \'DESKTOP-GNSLAB2.STELLA.LOCAL\\t\', \'Account_Name\': \'DESKTOP-GNSLAB2$\', \'Creator_Process_Name\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunkd.exe  \', \'Creator_Process_ID\': \' 0x728  \', \'EventCode\': \'4688\', \'New_Process_ID\': \'  0x14c8  \', \'New_Process_Name\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunk-winprintmon.exe  \', \'OriginatingComputerName\': \'10.0.5.112\\t\', \'RecordNumber\': \'968797\\t\', \'_time\': \'2020-05-08 11:01:11\', \'Process_Command_Line\': \' "C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunk-winprintmon.exe"  \', \'custodio_time\': 1588935671000, \'path\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunk-winprintmon.exe  \', \'Title\': [\'splunk-winprintmon.exe\']}']]

In [49]:
# add nodes to neo4j
graph.delete_all()
for elements in graph_nodelist:
    label =  elements['d']['labels'][0]

    props = {
        "id" : elements['id'],
        "t" : elements['t'],
        "mitre-attack" : [x for x in elements['d']['mitre-attack']],
        "tag" : elements['tag']
    }
    new_node = Node(label, **props)
    graph.create(new_node)

In [52]:
# add relationship to neo4j
for elements in graph_linklist:
    
    mathcerinit = NodeMatcher(graph)
    node1 = mathcerinit.match().where(id=elements['id1']).first()
    node2 = mathcerinit.match().where(id=elements['id2']).first()
    
    properties = {k: str(v) for x in graph_properties for k,v in x.items() if x['id'] == elements['id']}
    properties.update({"mitre-attack": [x for x in elements['d']['mitre-attack']],
                      "id1" : elements['id1'],
                       "id2" : elements['id2'],
                       "dt" : elements['dt']
                      })
    newnodelink = Relationship(node1, elements['t'], node2, **properties)
    graph.create(newnodelink)

AttributeError: 'list' object has no attribute 'items'

In [79]:
graphdata2 = []

# get node and convert back to graph struct
mathcerinit = NodeMatcher(graph)
nodefound = mathcerinit.match()
for elements in nodefound:
    id = elements['id']
    title=elements['t']
    labelslist = list(elements.labels)
    mitrelist = elements['mitre-attack']
    obj = create_node(id, title, labelslist, mitrelist)
    graphdata2.append(obj)

[{'id': '1',
  'parentId': ' ',
  'type': 'node',
  't': 'servicehost.exe',
  'd': {'labels': ['process'],
   'score_ttp': ' ',
   'mitre-attack': ['T1045'],
   'ids': []},
  'tag': 0},
 {'id': '2',
  'parentId': ' ',
  'type': 'node',
  't': 'fisher.exe',
  'd': {'labels': ['process'],
   'score_ttp': ' ',
   'mitre-attack': ['T1045'],
   'ids': []},
  'tag': 0}]

In [80]:
# get relationship and convert back to graph struct
mathcerinit = RelationshipMatcher(graph)
nodefound = mathcerinit.match()
for elements in nodefound:
    id = elements['id']
    datetimelist = elements['dt']
    id1 = elements['id1']
    id2 = elements['id2']
    linkname = type(elements).__name__
    propertyidlist = [id]

    obj = create_link(id, datetimelist, id1, id2, linkname, propertyidlist, mitrelist)
    graphdata2.append(obj)

[{'id': '1',
  'parentId': ' ',
  'type': 'node',
  't': 'servicehost.exe',
  'd': {'labels': ['process'],
   'score_ttp': ' ',
   'mitre-attack': ['T1045'],
   'ids': []},
  'tag': 0},
 {'id': '2',
  'parentId': ' ',
  'type': 'node',
  't': 'fisher.exe',
  'd': {'labels': ['process'],
   'score_ttp': ' ',
   'mitre-attack': ['T1045'],
   'ids': []},
  'tag': 0},
 {'dt': [12222, 212121],
  'a2': 'true',
  'id1': '1',
  'id2': '2',
  'id': '1',
  'type': 'link',
  't': 'created',
  'd': {'labels': [],
   'score_ttp': ' ',
   'mitre-attack': ['T1045'],
   'ids': ['1']}}]

In [123]:
import ast
graph_properties = []
# get properties and convert back to graph struct
mathcerinit = RelationshipMatcher(graph)
nodefound = mathcerinit.match()
for elements in nodefound:
    props = elements['Properties']
    temp = ast.literal_eval(props)
    
    temp['id'] = elements['id']
    graph_properties.append(tempdoct)

['{\'ComputerName\': \'DESKTOP-GNSLAB2.STELLA.LOCAL\\t\', \'Account_Name\': \'DESKTOP-GNSLAB2$\', \'Creator_Process_Name\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunkd.exe  \', \'Creator_Process_ID\': \' 0x728  \', \'EventCode\': \'4688\', \'New_Process_ID\': \'  0x14c8  \', \'New_Process_Name\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunk-winprintmon.exe  \', \'OriginatingComputerName\': \'10.0.5.112\\t\', \'RecordNumber\': \'968797\\t\', \'_time\': \'2020-05-08 11:01:11\', \'Process_Command_Line\': \' "C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunk-winprintmon.exe"  \', \'custodio_time\': 1588935671000, \'path\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunk-winprintmon.exe  \', \'Title\': [\'splunk-winprintmon.exe\']}']

In [124]:
graphdata2 = create_graphdata(graphdata2)
data = [
    graphdata1,
    graph_properties
]
data

[{'_id': '000',
  'type': 'graphData',
  'data': {'type': 'LinkChart',
   'items': [{'id': '1',
     'parentId': ' ',
     'type': 'node',
     't': 'servicehost.exe',
     'd': {'labels': ['process'],
      'score_ttp': ' ',
      'mitre-attack': ['T1045'],
      'ids': []},
     'tag': 0},
    {'id': '2',
     'parentId': ' ',
     'type': 'node',
     't': 'fisher.exe',
     'd': {'labels': ['process'],
      'score_ttp': ' ',
      'mitre-attack': ['T1045'],
      'ids': []},
     'tag': 0},
    {'dt': [12222, 212121],
     'a2': 'true',
     'id1': '1',
     'id2': '2',
     'id': '1',
     'type': 'link',
     't': 'created',
     'd': {'labels': [],
      'score_ttp': ' ',
      'mitre-attack': ['T1022'],
      'ids': ['101']}}]}},
 ['{\'ComputerName\': \'DESKTOP-GNSLAB2.STELLA.LOCAL\\t\', \'Account_Name\': \'DESKTOP-GNSLAB2$\', \'Creator_Process_Name\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunkd.exe  \', \'Creator_Process_ID\': \' 0x728  \', \'EventCod

# MONGODB

In [38]:
client = MongoClient("mongodb://localhost:27017")
DB = client['DB']

In [39]:
gdata = DB.get_collection('Graphdata')

In [40]:
gdata.insert_one({'case': 'spf1', 'graphData': fakegraphdata})

In [54]:
getdata = gdata.find_one({'case': 'spf1'})
data = getdata['graphData']
data

[{'_id': '000',
  'type': 'graphData',
  'data': {'type': 'LinkChart',
   'items': [{'id': '1',
     'parentId': ' ',
     'type': 'node',
     't': 'servicehost.exe',
     'd': {'labels': ['process'],
      'score_ttp': ' ',
      'mitre-attack': ['T1045'],
      'ids': []},
     'tag': 0},
    {'id': '2',
     'parentId': ' ',
     'type': 'node',
     't': 'fisher.exe',
     'd': {'labels': ['process'],
      'score_ttp': ' ',
      'mitre-attack': ['T1045'],
      'ids': []},
     'tag': 0},
    {'dt': [12222, 212121],
     'a2': 'true',
     'id1': '1',
     'id2': '2',
     'id': '1',
     'type': 'link',
     't': 'created',
     'd': {'labels': [],
      'score_ttp': ' ',
      'mitre-attack': ['T1022'],
      'ids': ['101']}}]}},
 ['{\'ComputerName\': \'DESKTOP-GNSLAB2.STELLA.LOCAL\\t\', \'Account_Name\': \'DESKTOP-GNSLAB2$\', \'Creator_Process_Name\': \' C:\\\\Program Files\\\\SplunkUniversalForwarder\\\\bin\\\\splunkd.exe  \', \'Creator_Process_ID\': \' 0x728  \', \'EventCod